# Marca e Recaptura

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [3]:
from utils import set_pyplot_params
set_pyplot_params()

Este capítulo introduz experiências de "marcar e recapturar", nas quais se recolhem amostras de indivíduos de uma população, marcamo-los de alguma forma, e depois retiramos uma segunda amostra da mesma população.  Vendo quantos indivíduos da segunda amostra são marcados, podemos estimar o tamanho da população.

Experiências como esta foram originalmente utilizadas na ecologia, mas revelaram-se úteis em muitos outros campos.  Exemplos neste capítulo incluem a engenharia de software e a epidemiologia.

Também, neste capítulo vamos trabalhar com modelos que têm três parâmetros, por isso vamos alargar as distribuições conjuntas que temos vindo a utilizar a três dimensões.

Mas primeiro, os ursos pardos.

## O problema do urso pardo

Em 1996 e 1997, investigadores instalaram armadilhas para ursos em locais na Colômbia Britânica e Alberta, Canadá, num esforço para estimar a população de ursos pardos.  Eles descrevem a experiência em [this article](https://www.researchgate.net/publication/229195465_Estimating_Population_Size_of_Grizzly_Bears_Using_Hair_Capture_DNA_Profiling_and_Mark-Recapture_Analysis).

A "armadilha" consiste numa isca e vários fios de arame farpado destinados a capturar amostras de cabelo de ursos que visitam a isca.  Utilizando as amostras de cabelo, os investigadores utilizam a análise de ADN para identificar ursos individuais.

Durante a primeira sessão, os investigadores colocaram armadilhas em 76 locais.  Voltando 10 dias depois, obtiveram 1043 amostras de cabelo e identificaram 23 ursos diferentes.  Durante uma segunda sessão de 10 dias, obtiveram 1191 amostras de 19 ursos diferentes, onde 4 dos 19 eram de ursos que tinham identificado no primeiro lote.

Para estimar a população de ursos a partir destes dados, precisamos de um modelo para a probabilidade de cada urso ser observado durante cada sessão.  Como ponto de partida, faremos a hipótese mais simples, de que cada urso da população tem a mesma probabilidade (desconhecida) de ser amostrado durante cada sessão.

Com estes pressupostos podemos calcular a probabilidade dos dados para uma gama de populações possíveis.

Como exemplo, suponhamos que a população real de ursos é de 100.

Após a primeira sessão, 23 dos 100 ursos foram identificados.

Durante a segunda sessão, se escolhermos 19 ursos ao acaso, qual é a probabilidade de 4 deles terem sido previamente identificados?

Vou definir

* $N$: tamanho real da população, 100.

* $K$: número de ursos identificados na primeira sessão, 23.

* $n$: número de ursos observados na segunda sessão, 19 no exemplo.

* $k$: número de ursos na segunda sessão que foram previamente identificados, 4.

Para valores dados de $N$, $K$, e $n$, a probabilidade de encontrar ursos previamente identificados é dada pelo [hypergeometric distribution](https://en.wikipedia.org/wiki/Hypergeometric_distribution):

$$$binom{K}{k} \binom{N-K}{n-k}/ \binom{N}{n}$$

onde o [binomial coefficient](https://en.wikipedia.org/wiki/Binomial_coefficient), $\binom{K}{k}$, é o número de subconjuntos de tamanho $k$ que podemos escolher entre uma população de tamanho $K$.

Para compreender porquê, considere: 

* O denominador, $n$, é o número de subconjuntos de $n$ que poderíamos escolher entre uma população de ursos de $N$.

* O numerador é o número de subconjuntos que contêm $k$ ursos dos $K$ previamente identificados e $n-k$ dos $N-K$ anteriormente não vistos.

SciPy fornece `hipergeom`, que podemos utilizar para calcular esta probabilidade para uma gama de valores de $k$.

In [4]:
import numpy as np
from scipy.stats import hypergeom

N = 100
K = 23
n = 19

ks = np.arange(12)
ps = hypergeom(N, K, n).pmf(ks)

O resultado é a distribuição de $k$ com determinados parâmetros $N$, $K$, e $n$.

Aqui está o que parece.

In [5]:
import matplotlib.pyplot as plt
from utils import decorate

plt.bar(ks, ps)

decorate(xlabel='Number of bears observed twice',
         ylabel='PMF',
         title='Hypergeometric distribution of k (known population 100)')

O valor mais provável de $k$ é 4, que é o valor efectivamente observado na experiência.  

Isto sugere que $N=100$ é uma estimativa razoável da população, tendo em conta estes dados.

Calculámos a distribuição de $k$ dado $N$, $K$, e $n$.

Agora vamos pelo outro caminho: dado $K$, $n$ e $k$, como podemos estimar a população total, $N$?

## A Actualização

Como ponto de partida, vamos supor que, antes deste estudo, um perito estima que a população de ursos local se situa entre 50 e 500, e igualmente provável que seja qualquer valor nessa faixa.

Vou utilizar o `make_uniform` para fazer uma distribuição uniforme de inteiros nesta gama.

In [6]:
import numpy as np
from utils import make_uniform

qs = np.arange(50, 501)
prior_N = make_uniform(qs, name='N')
prior_N.shape

Portanto, esse é o nosso anterior.

Para calcular a probabilidade dos dados, podemos utilizar "hypergeom" com constantes "K" e "n", e uma gama de valores de "N". 

In [7]:
Ns = prior_N.qs
K = 23
n = 19
k = 4

likelihood = hypergeom(Ns, K, n).pmf(k)

Podemos calcular o posterior da forma habitual.

In [8]:
posterior_N = prior_N * likelihood
posterior_N.normalize()

E aqui está o que parece.

In [9]:
posterior_N.plot(color='C4')

decorate(xlabel='Population of bears (N)',
         ylabel='PDF',
         title='Posterior distribution of N')

O valor mais provável é 109.

In [10]:
posterior_N.max_prob()

Mas a distribuição é enviesada para a direita, pelo que a média posterior é substancialmente mais elevada.

In [11]:
posterior_N.mean()

E o intervalo credível é bastante amplo.

In [12]:
posterior_N.credible_interval(0.9)

Esta solução é relativamente simples, mas afinal podemos fazer um pouco melhor se modelarmos explicitamente a probabilidade desconhecida de observar um urso.

## Modelo de Dois Parâmetros

A seguir vamos tentar um modelo com dois parâmetros: o número de ursos, `N`, e a probabilidade de observar um urso, `p`.

Vamos assumir que a probabilidade é a mesma em ambas as rondas, o que é provavelmente razoável neste caso, porque é o mesmo tipo de armadilha no mesmo local.

Vamos também assumir que as probabilidades são independentes; isto é, a probabilidade de um urso ser observado na segunda ronda não depende de ter sido observado na primeira ronda.  Esta suposição pode ser menos razoável, mas por agora é uma simplificação necessária.

Aqui estão novamente as contagens:

In [13]:
K = 23
n = 19
k = 4

Para este modelo, vou expressar os dados numa notação que facilitará a generalização para mais de duas rondas: 

* `k10` é o número de ursos observados na primeira ronda mas não na segunda,

* k01" é o número de ursos observados na segunda ronda, mas não na primeira, e

* `k11` é o número de ursos observados em ambas as rondas.

Aqui estão os seus valores.

In [14]:
k10 = 23 - 4
k01 = 19 - 4
k11 = 4

Suponhamos que conhecemos os valores reais de `N` e `p`.  Podemos utilizá-los para calcular a probabilidade destes dados.

Por exemplo, suponhamos que sabemos que `N=100` e `p=0,2`.

Podemos utilizar `N` para calcular `k00`, que é o número de ursos não observados.

In [15]:
N = 100

observed = k01 + k10 + k11
k00 = N - observed
k00

Para a actualização, será conveniente armazenar os dados como uma lista que represente o número de ursos em cada categoria.

In [16]:
x = [k00, k01, k10, k11]
x

Agora, se soubermos `p=0,2`, podemos calcular a probabilidade de um urso cair em cada categoria.  Por exemplo, a probabilidade de ser observado em ambas as rondas é `p*p`, e a probabilidade de não ser observado em ambas as rondas é `q*q` (onde `q=1-p`).

In [17]:
p = 0.2
q = 1-p
y = [q*q, q*p, p*q, p*p]
y

Agora a probabilidade dos dados é dada pelo [multinomial distribution](https://en.wikipedia.org/wiki/Multinomial_distribution):

$$$frac{N!}{N!}prod x_i!} \y_i^{x_i}$$

onde $N$ é a população real, $x$ é uma sequência com as contagens em cada categoria, e $y$ é uma sequência de probabilidades para cada categoria.

SciPy fornece `multinomial', que fornece `pmf', que calcula esta probabilidade.

Aqui está a probabilidade dos dados para estes valores de `N` e `p`.

In [18]:
from scipy.stats import multinomial

likelihood = multinomial.pmf(x, N, y)
likelihood

Essa é a probabilidade se soubermos 'N' e 'P', mas claro que não sabemos.  Assim, escolheremos distribuições anteriores de `N` e `p`, e utilizaremos as probabilidades para a actualizar. 

## O Prior

Voltaremos a utilizar `prior_N` para a distribuição prévia de `N`, e um anterior uniforme para a probabilidade de observar um urso, `p`:

In [19]:
qs = np.linspace(0, 0.99, num=100)
prior_p = make_uniform(qs, name='p')

Podemos fazer uma distribuição conjunta da forma habitual.

In [20]:
from utils import make_joint

joint_prior = make_joint(prior_p, prior_N)
joint_prior.shape

O resultado é um Pandas "DataFrame" com valores de "N" ao longo das linhas e valores de "p" através das colunas.

Contudo, para este problema será conveniente representar a distribuição prévia como uma "Série 1-D" em vez de uma "Série de Dados 2-D".

Podemos converter de um formato para o outro utilizando o `stack`.

In [21]:
from empiricaldist import Pmf

joint_pmf = Pmf(joint_prior.stack())
joint_pmf.head(3)

In [22]:
type(joint_pmf)

In [23]:
type(joint_pmf.index)

In [24]:
joint_pmf.shape

O resultado é um `Pmf` cujo índice é um `MultiIndex`.

Um `MultiIndex` pode ter mais de uma coluna; neste exemplo, a primeira coluna contém valores de `N` e a segunda coluna contém valores de `p`.

O `Pmf` tem uma linha (e uma probabilidade prévia) para cada possível par de parâmetros `N` e `p`.

Assim, o número total de filas é o produto dos comprimentos "prior_N" e "prior_p".

Agora temos de calcular a probabilidade dos dados para cada par de parâmetros.

## A Actualização

Para atribuir espaço para as probabilidades, é conveniente fazer uma cópia do `joint_pmf`:

In [25]:
likelihood = joint_pmf.copy()

À medida que fazemos um loop através dos pares de parâmetros, calculamos a probabilidade dos dados como na secção anterior, e depois armazenamos o resultado como um elemento de "probabilidade".

In [26]:
observed = k01 + k10 + k11

for N, p in joint_pmf.index:
    k00 = N - observed
    x = [k00, k01, k10, k11]
    q = 1-p
    y = [q*q, q*p, p*q, p*p]
    likelihood[N, p] = multinomial.pmf(x, N, y)

Agora podemos calcular o posterior da forma habitual.

In [27]:
posterior_pmf = joint_pmf * likelihood
posterior_pmf.normalize()

Vamos utilizar novamente o `plot_contour` para visualizar a distribuição posterior conjunta.

Mas lembrem-se que a distribuição posterior que acabámos de calcular é representada como um `Pmf', que é uma `Série', e o `plot_contour' espera um `DataFrame'.

Uma vez que utilizámos o "ataque" para converter de um "DataFrame" para uma "Série", podemos utilizar o "ataque" para ir para o outro lado.

In [28]:
joint_posterior = posterior_pmf.unstack()

E o resultado é o seguinte.

In [29]:
from utils import plot_contour

plot_contour(joint_posterior)

decorate(title='Joint posterior distribution of N and p')

Os valores mais prováveis de `N` são próximos de 100, como no modelo anterior. Os valores mais prováveis de `p` são próximos de 0,2.

A forma deste contorno indica que estes parâmetros estão correlacionados.  Se `p` estiver perto do limite inferior da gama, os valores mais prováveis de `N` são superiores; se `p` estiver perto do limite superior da gama, `N` é inferior. 

Agora que temos um 'DataFrame' posterior, podemos extrair as distribuições marginais da forma habitual.

In [30]:
from utils import marginal

posterior2_p = marginal(joint_posterior, 0)
posterior2_N = marginal(joint_posterior, 1)

Aqui está a distribuição posterior para `p`:

In [31]:
posterior2_p.plot(color='C1')

decorate(xlabel='Probability of observing a bear',
         ylabel='PDF',
         title='Posterior marginal distribution of p')

Os valores mais prováveis são próximos de 0,2.

Aqui está a distribuição posterior de `N` com base no modelo de dois parâmetros, juntamente com a posterior que obtivemos utilizando o modelo de um parâmetro (hipergeométrico).

In [32]:
posterior_N.plot(label='one-parameter model', color='C4')
posterior2_N.plot(label='two-parameter model', color='C1')

decorate(xlabel='Population of bears (N)',
         ylabel='PDF',
         title='Posterior marginal distribution of N')

Com o modelo de dois parâmetros, a média é um pouco mais baixa e o intervalo credível de 90% é um pouco mais estreito.

In [33]:
print(posterior_N.mean(), 
      posterior_N.credible_interval(0.9))

In [34]:
print(posterior2_N.mean(), 
      posterior2_N.credible_interval(0.9))

O modelo de dois parâmetros produz uma distribuição posterior mais estreita para `N`, em comparação com o modelo de um parâmetro, porque tira partido de uma fonte adicional de informação: a consistência das duas observações.

Para ver como isto ajuda, considere um cenário em que "N" é relativamente baixo, como 138 (a média posterior do modelo de dois parâmetros).

In [35]:
N1 = 138

Dado que vimos 23 ursos durante o primeiro julgamento e 19 durante o segundo, podemos estimar o valor correspondente de `p`.

In [36]:
mean = (23 + 19) / 2
p = mean/N1
p

Com estes parâmetros, quanta variabilidade espera no número de ursos de um ensaio para o outro?  Podemos quantificar isso através do cálculo do desvio padrão da distribuição binomial com estes parâmetros.

In [37]:
from scipy.stats import binom

binom(N1, p).std()

Agora vamos considerar um segundo cenário onde `N` é 173, a média posterior do modelo de um parámetro.  O valor correspondente de `p` é mais baixo.

In [38]:
N2 = 173
p = mean/N2
p

Neste cenário, a variação que esperamos ver de um julgamento para outro é maior.

In [39]:
binom(N2, p).std()

Assim, se o número de ursos que observamos for o mesmo em ambos os ensaios, isso seria prova de valores mais baixos de `N`, onde esperamos mais consistência.

Se o número de ursos for substancialmente diferente entre as duas provas, isso seria prova de valores mais elevados de `N`.

Nos dados reais, a diferença entre os dois ensaios é baixa, razão pela qual a média posterior do modelo de dois parâmetros é inferior.

O modelo de dois parâmetros tira partido de informação adicional, razão pela qual o intervalo credível é mais estreito.

## Distribuições conjuntas e marginais

As distribuições marginais são chamadas "marginais" porque, numa visualização comum, aparecem nas margens da parcela.

Seaborn fornece uma classe chamada `JointGrid` que cria esta visualização.

A função seguinte utiliza-a para mostrar as distribuições conjuntas e marginais numa única parcela.

In [40]:
import pandas as pd
from seaborn import JointGrid

def joint_plot(joint, **options):
    """Show joint and marginal distributions.
    
    joint: DataFrame that represents a joint distribution
    options: passed to JointGrid
    """
    # get the names of the parameters
    x = joint.columns.name
    x = 'x' if x is None else x

    y = joint.index.name
    y = 'y' if y is None else y

    # make a JointGrid with minimal data
    data = pd.DataFrame({x:[0], y:[0]})
    g = JointGrid(x=x, y=y, data=data, **options)

    # replace the contour plot
    g.ax_joint.contour(joint.columns, 
                       joint.index, 
                       joint, 
                       cmap='viridis')
    
    # replace the marginals
    marginal_x = marginal(joint, 0)
    g.ax_marg_x.plot(marginal_x.qs, marginal_x.ps)
    
    marginal_y = marginal(joint, 1)
    g.ax_marg_y.plot(marginal_y.ps, marginal_y.qs)

In [41]:
joint_plot(joint_posterior)

Uma `JointGrid` é uma forma concisa de representar visualmente as distribuições conjuntas e marginais.

## O problema do Índice Lincoln

Em [an excellent blog post](http://www.johndcook.com/blog/2010/07/13/lincoln-index/), John D. Cook escreveu sobre o índice Lincoln, que é uma forma de estimar o

número de erros num documento (ou programa), comparando os resultados de

dois provadores independentes.

Aqui está a sua apresentação do problema:

> "Suponha que tem um testador que encontra 20 bugs no seu programa. Você

> querem estimar quantos bugs estão realmente no programa. Sabe que

> existem pelo menos 20 bugs, e se tiver a confiança suprema no seu

> testador, pode supor que existem cerca de 20 insectos. Mas talvez o seu

> testador não é muito bom. Talvez haja centenas de insectos. Como pode

> tem alguma ideia de quantos insectos existem? Não há maneira de saber com um

> testador. Mas se tiver dois testadores, pode ter uma boa ideia, mesmo que

> não se sabe quão habilidosos são os provadores".

Suponha que o primeiro verificador encontra 20 insectos, o segundo encontra 15, e eles

encontrar 3 em comum; como podemos estimar o número de bugs?

Este problema é semelhante ao problema do urso pardo, por isso vou representar os dados da mesma forma.

In [42]:
k10 = 20 - 3
k01 = 15 - 3
k11 = 3

Mas neste caso provavelmente não é razoável presumir que os testadores têm a mesma probabilidade de encontrar um bug.

Assim, vou definir dois parâmetros, `p0` para a probabilidade de o primeiro testador encontrar um bug, e `p1` para a probabilidade de o segundo testador encontrar um bug.

Continuarei a assumir que as probabilidades são independentes, o que é como assumir que todos os insectos são igualmente fáceis de encontrar.  Isso pode não ser uma boa suposição, mas vamos continuar por agora.

Como exemplo, suponhamos que sabemos que as probabilidades são de 0,2 e 0,15.

In [43]:
p0, p1 = 0.2, 0.15

Podemos calcular o conjunto de probabilidades, `y`, desta forma:

In [44]:
def compute_probs(p0, p1):
    """Computes the probability for each of 4 categories."""
    q0 = 1-p0
    q1 = 1-p1
    return [q0*q1, q0*p1, p0*q1, p0*p1]

In [45]:
y = compute_probs(p0, p1)
y

Com estas probabilidades, existe um 

68% de hipóteses de que nenhum dos testadores encontre o insecto e um

3% de hipóteses que ambos têm. 

Fingindo que estas probabilidades são conhecidas, podemos calcular a distribuição posterior para `N`.

Aqui está uma distribuição prévia que é uniforme de 32 a 350 insectos.

In [46]:
qs = np.arange(32, 350, step=5) 
prior_N = make_uniform(qs, name='N')
prior_N.head(3)

Colocarei os dados numa matriz, com 0 como place-keeper para o valor desconhecido `k00`.

In [47]:
data = np.array([0, k01, k10, k11])

E aqui estão as probabilidades para cada valor de `N`, com `ps` como uma constante.

In [48]:
likelihood = prior_N.copy()
observed = data.sum()
x = data.copy()

for N in prior_N.qs:
    x[0] = N - observed
    likelihood[N] = multinomial.pmf(x, N, y)

Podemos calcular o posterior da forma habitual.

In [49]:
posterior_N = prior_N * likelihood
posterior_N.normalize()

E aqui está o que parece.

In [50]:
posterior_N.plot(color='C4')

decorate(xlabel='Number of bugs (N)',
         ylabel='PMF',
         title='Posterior marginal distribution of n with known p1, p2')

In [51]:
print(posterior_N.mean(), 
      posterior_N.credible_interval(0.9))

Com o pressuposto de que `p0` e `p1` são conhecidos por `0,2` e `0,15`, a média posterior é 102 com intervalo credível de 90% (77, 127).

Mas este resultado baseia-se no pressuposto de que conhecemos as probabilidades, e não conhecemos.

## Modelo de três parâmetros

O que precisamos é de um modelo com três parâmetros: `N`, `p0`, e `p1`.

Utilizaremos novamente o `prior_N` para a distribuição prévia de `N`, e aqui estão os antecedentes para `p0` e `p1`:

In [52]:
qs = np.linspace(0, 1, num=51)
prior_p0 = make_uniform(qs, name='p0')
prior_p1 = make_uniform(qs, name='p1')

Agora temos de os montar num anterior conjunto com três dimensões.

Vou começar por colocar os dois primeiros num "DataFrame".

In [53]:
joint2 = make_joint(prior_p0, prior_N)
joint2.shape

Agora vou empilhá-los, como no exemplo anterior, e colocar o resultado num `Pmf`.

In [54]:
joint2_pmf = Pmf(joint2.stack())
joint2_pmf.head(3)

Podemos utilizar o `make_joint` novamente para adicionar no terceiro parâmetro.

In [55]:
joint3 = make_joint(prior_p1, joint2_pmf)
joint3.shape

O resultado é um "DataFrame" com valores de `N` e `p0` num `MultiIndex` que desce as linhas e valores de `p1` num índice que atravessa as colunas.

In [56]:
joint3.head(3)

Agora vou voltar a aplicar o `stack`:

In [57]:
joint3_pmf = Pmf(joint3.stack())
joint3_pmf.head(3)

O resultado é um `Pmf` com uma coluna de três colunas `MultiIndex` contendo todos os possíveis trigémeos de parâmetros.

O número de filas é o produto do número de valores nos três antecedentes, que é de quase 170.000.

In [58]:
joint3_pmf.shape

Isso ainda é suficientemente pequeno para ser prático, mas levará mais tempo a calcular as probabilidades do que nos exemplos anteriores.

Eis o laço que calcula as probabilidades; é semelhante ao da secção anterior:

In [59]:
likelihood = joint3_pmf.copy()
observed = data.sum()
x = data.copy()

for N, p0, p1 in joint3_pmf.index:
    x[0] = N - observed
    y = compute_probs(p0, p1)
    likelihood[N, p0, p1] = multinomial.pmf(x, N, y)

Podemos calcular o posterior da forma habitual.

In [60]:
posterior_pmf = joint3_pmf * likelihood
posterior_pmf.normalize()

Agora, para extrair as distribuições marginais, podíamos desempilhar a junta posterior como fizemos na secção anterior.

Mas `Pmf` fornece uma versão de `marginal` que funciona com um `Pmf` em vez de um `DataFrame`.

Eis como o utilizamos para obter a distribuição posterior do `N`.

In [61]:
posterior_N = posterior_pmf.marginal(0)

E aqui está o que parece.

In [62]:
posterior_N.plot(color='C4')

decorate(xlabel='Number of bugs (N)',
         ylabel='PDF',
         title='Posterior marginal distributions of N')

In [63]:
posterior_N.mean()

A média posterior é 105 insectos, o que sugere que ainda há muitos insectos que os testadores não encontraram.

Aqui estão os cartazes para `p0` e `p1`.

In [64]:
posterior_p1 = posterior_pmf.marginal(1)
posterior_p2 = posterior_pmf.marginal(2)

posterior_p1.plot(label='p1')
posterior_p2.plot(label='p2')

decorate(xlabel='Probability of finding a bug',
         ylabel='PDF',
         title='Posterior marginal distributions of p1 and p2')

In [65]:
posterior_p1.mean(), posterior_p1.credible_interval(0.9)

In [66]:
posterior_p2.mean(), posterior_p2.credible_interval(0.9)

Comparando as distribuições posteriores, o testador que encontrou mais insectos tem provavelmente uma maior probabilidade de encontrar insectos.  Os meios posteriores são cerca de 23% e 18%.  Mas as distribuições sobrepõem-se, pelo que não devemos estar demasiado seguros.

Este é o primeiro exemplo que vimos com três parâmetros.

À medida que o número de parâmetros aumenta, o número de combinações aumenta rapidamente.

O método que temos utilizado até agora, enumerando todas as combinações possíveis, torna-se impraticável se o número de parâmetros for superior a 3 ou 4.

No entanto, existem outros métodos que podem lidar com modelos com muitos mais parâmetros, como veremos em <<_MCMC>>>>.

## Resumo

Os problemas neste capítulo são exemplos de experiências [mark and recapture](https://en.wikipedia.org/wiki/Mark_and_recapture), que são utilizadas na ecologia para estimar as populações animais.  Também têm aplicações em engenharia, como no problema do índice Lincoln.  E nos exercícios verá que também são utilizados em epidemiologia.

Este capítulo introduz duas novas distribuições de probabilidade:

* A distribuição hipergeométrica é uma variação da distribuição binomial em que as amostras são retiradas da população sem substituição. 

* A distribuição multinomial é uma generalização da distribuição binomial onde existem mais de dois resultados possíveis.

Também neste capítulo, vimos o primeiro exemplo de um modelo com três parâmetros.  Veremos mais nos capítulos seguintes.

## Exercícios

**Exercise:** [In an excellent paper](http://chao.stat.nthu.edu.tw/wordpress/paper/110.pdf), Anne Chao explica como as experiências de marcação e recaptura são utilizadas na epidemiologia para estimar a prevalência de uma doença numa população humana com base em múltiplas listas incompletas de casos.

Um dos exemplos desse documento é um estudo "para estimar o número de pessoas que foram infectadas pela hepatite num surto que ocorreu num colégio no norte de Taiwan e nas suas imediações, de Abril a Julho de 1995".

Estavam disponíveis três listas de casos:

1. 135 casos identificados utilizando um teste de soro. 

2. 122 casos comunicados por hospitais locais. 

3. 126 casos relatados em questionários recolhidos por epidemiologistas.

Neste exercício, utilizaremos apenas as duas primeiras listas; no próximo exercício, traremos a terceira lista.

Fazer um conjunto prévio e actualizá-lo utilizando estes dados, depois calcular a média posterior de `N` e um intervalo credível de 90%.

A seguinte matriz contém 0 como detentor de lugar para o valor desconhecido de `k00`, seguido pelos valores conhecidos de `k01`, `k10`, e `k11`. 

In [67]:
data2 = np.array([0, 73, 86, 49])

Estes dados indicam que existem 73 casos na segunda lista que não figuram na primeira, 86 casos na primeira lista que não figuram na segunda, e 49 casos em ambas as listas.

Para manter as coisas simples, vamos assumir que cada caso tem a mesma probabilidade de aparecer em cada lista.  Assim, utilizaremos um modelo de dois parâmetros onde "N" é o número total de casos e "p" é a probabilidade de qualquer caso aparecer em qualquer lista.

Aqui estão os antecedentes com os quais pode começar (mas sinta-se à vontade para os modificar).

In [68]:
qs = np.arange(200, 500, step=5)
prior_N = make_uniform(qs, name='N')
prior_N.head(3)

In [69]:
qs = np.linspace(0, 0.98, num=50)
prior_p = make_uniform(qs, name='p')
prior_p.head(3)

In [70]:
# Solution goes here

In [71]:
# Solution goes here

In [72]:
# Solution goes here

In [73]:
# Solution goes here

In [74]:
# Solution goes here

In [75]:
# Solution goes here

In [76]:
# Solution goes here

In [77]:
# Solution goes here

**Exercício:** Agora vamos fazer a versão do problema com as três listas.  Aqui estão os dados do trabalho de Chou:

```
Hepatitis A virus list
P    Q    E    Data
1    1    1    k111 =28
1    1    0    k110 =21
1    0    1    k101 =17
1    0    0    k100 =69
0    1    1    k011 =18
0    1    0    k010 =55
0    0    1    k001 =63
0    0    0    k000 =??
```

Escrever um laço que calcula a probabilidade dos dados para cada par de parâmetros, depois actualizar o anterior e calcular a média posterior de `N`.  Como se compara com os resultados utilizando apenas as duas primeiras listas?

Aqui estão os dados numa matriz NumPy (em ordem inversa).

In [78]:
data3 = np.array([0, 63, 55, 18, 69, 17, 21, 28])

Mais uma vez, o primeiro valor é um place-keeper para o desconhecido `k000`.  O segundo valor é `k001`, o que significa que há 63 casos que aparecem na terceira lista, mas não os dois primeiros.  E o último valor é `k111`, o que significa que há 28 casos que aparecem nas três listas.

Na versão de duas listas do problema, calculámos `ps' enumerando as combinações de `p' e `q'.

In [79]:
q = 1-p
ps = [q*q, q*p, p*q, p*p]

Poderíamos fazer a mesma coisa para a versão de três listas, calculando a probabilidade para cada uma das oito categorias.  Mas podemos generalizá-la reconhecendo que estamos a computar o produto cartesiano de `p` e `q`, repetidos uma vez para cada lista.

E podemos utilizar a seguinte função (baseada em [this StackOverflow answer](https://stackoverflow.com/questions/58242078/cartesian-product-of-arbitrary-lists-in-pandas/58242079#58242079)) para calcular os produtos cartesianos:

In [80]:
def cartesian_product(*args, **options):
    """Cartesian product of sequences.
    
    args: any number of sequences
    options: passes to `MultiIndex.from_product`
    
    returns: DataFrame with one column per sequence
    """
    index = pd.MultiIndex.from_product(args, **options)
    return pd.DataFrame(index=index).reset_index()

Aqui está um exemplo com `p=0,2`:

In [81]:
p = 0.2
t = (1-p, p)
df = cartesian_product(t, t, t)
df

Para calcular a probabilidade para cada categoria, levamos o produto através das colunas:

In [82]:
y = df.prod(axis=1)
y

Agora termina a partir daí.

In [83]:
# Solution goes here

In [84]:
# Solution goes here

In [85]:
# Solution goes here

In [86]:
# Solution goes here

In [87]:
# Solution goes here

In [88]:
# Solution goes here

In [89]:
# Solution goes here

In [90]:
# Solution goes here